In [2]:
import pandas as pd
import numpy as np
import math
import requests
import geopandas as gpd

liste_med_ban = pd.read_csv('liste_med_ban.csv')

/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_10862/1399691685.py:7: DtypeWarning: Columns (28,29,30,31,32,35,41,42,43,44,45,46,50) have mixed types. Specify dtype option on import or set low_memory=False.
  liste_med_ban = pd.read_csv('liste_med_ban.csv')


# Accéder à la distance par requête API

In [3]:
def cp_et_nom(cp,nom):
    if isinstance(cp,int) and isinstance(nom,str):
        return (str(cp)+' '+nom)
    else:
        return np.nan

In [4]:
liste_med_ban['cp_ville'] = liste_med_ban.apply(lambda x: cp_et_nom(x['codepostalcoordstructure'],x['libellécommunecoordstructure']),axis=1)

In [5]:
def transfo_adresse(CodePostal,NumeroVoie,NomVoie):
    num = (int(NumeroVoie))
    return (str(num)+', '+str(NomVoie)+', '+str(CodePostal))

liste_med_ban['adresse'] = liste_med_ban.apply(lambda x: transfo_adresse(x['cp_ville'],x['numérovoiecoordstructure'],x['Voie']),axis=1)

In [6]:
API_URL = "https://api-adresse.data.gouv.fr/{}/"

def req_to(endpoint: str, params: dict | None = None, files: dict | None = None) -> dict:
    url = API_URL.format(endpoint)

    if not files:
        response = requests.get(url, params=params)

    else:
        response = requests.post(url, files=files, data=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        raise Exception(f"Request failed with code : {response.status_code}")


def get_geocode(**kwargs) -> dict:
    if not 'q' in kwargs:
        raise ValueError('Il manque un argument obligatoire : q')
    
    return req_to("search", kwargs)



In [7]:
numero=str(int(liste_med_ban['numérovoiecoordstructure'].iloc[0]))
voie = liste_med_ban['Voie'].iloc[0]
cp_ville=liste_med_ban['cp_ville'].iloc[0]
adresse = numero+', '+voie+', '+cp_ville
print(adresse)

# Requête simple directe:
res = get_geocode(q = adresse)
print(res)

52, rue jean jaures, 97110 Pointe-à-Pitre
{'type': 'FeatureCollection', 'version': 'draft', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-61.536772, 16.23989]}, 'properties': {'label': '52 Rue Jean Jaures 97110 Pointe-à-Pitre', 'score': 0.9688018181818181, 'housenumber': '52', 'id': '97120_0496_00052', 'banId': '3565ae2f-5cbf-4aab-8543-7a3fda9b9059', 'name': '52 Rue Jean Jaures', 'postcode': '97110', 'citycode': '97120', 'x': 656379.35, 'y': 1796029.48, 'city': 'Pointe-à-Pitre', 'context': '971, Guadeloupe', 'type': 'housenumber', 'importance': 0.65682, 'street': 'Rue Jean Jaures'}}], 'attribution': 'BAN', 'licence': 'ETALAB-2.0', 'query': '52, rue jean jaures, 97110 Pointe-à-Pitre', 'limit': 5}


In [8]:
def distance_long_lat(lat1, lon1, lat2, lon2):
    # Rayon de la Terre en km
    R = 6371.0
    
    # Conversion des degrés en radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Calcul des différences
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad
    
    # Formule de Haversine
    a = math.sin(delta_lat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # Distance
    distance = R * c
    
    return distance

# Exemple d'utilisation
lat1, lon1 = 48.8566, 2.3522  # Paris
lat2, lon2 = 34.0522, -118.2437  # Los Angeles

distance = distance_long_lat(lat1, lon1, lat2, lon2)
print(f"La distance entre Paris et Los Angeles est d'environ {distance:.2f} km.")

La distance entre Paris et Los Angeles est d'environ 9085.51 km.


In [9]:
def recup_long(adr):
    if isinstance(adr,str):
        try:
            return(get_geocode(q=adr)['features'][0]['geometry']['coordinates'][0])
        except (IndexError, KeyError, TypeError):
            print(adr)
            return None
    else :
        return (np.nan)
    
def recup_lat(adr):
    if isinstance(adr,str):
        try:
            return(get_geocode(q=adr)['features'][0]['geometry']['coordinates'][1])
        except (IndexError, KeyError, TypeError):
            print(adr)
            return None
    else :
        return (np.nan)
liste_med_ban['longitude API'] = liste_med_ban['adresse'].apply(recup_long)

#liste_med_ban['latitude API'] = get_geocode(q = liste_med_ban['adresse'])['features'][0]['geometry']['coordinates'][1]

#liste_med_ban['adresse','longitude API','latitude API']

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_10862/4230522017.py", line 20, in <module>
    liste_med_ban['longitude API'] = liste_med_ban['adresse'].apply(recup_long)
                                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/pandas/core/series.py", line 4924, in apply
    ).apply()
      ^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/pandas/core/apply.py", line 1427, in apply
    return self.apply_standard()
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/pandas/core/apply.py", line 1507, in apply_standard
    mapped = obj._map_values(
             ^^^^^^^^^^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11

# Utilisation base de données centroides

In [10]:
gdf = gpd.read_file('data/a-com2022-topo-2154.json')
gdf.head()

,id,codgeo,dep,reg,xcl2154,ycl2154,libgeo,geometry
0,None,01001,01,84,848241,6563021,L'Abergement-Clémenciat,"POLYGON ((846981.436 6564107.360, 847017.457 6..."
1,None,01002,01,84,887495,6548152,L'Abergement-de-Varey,"POLYGON ((886027.651 6548170.900, 886157.325 6..."
2,None,01004,01,84,882724,6542583,Ambérieu-en-Bugey,"POLYGON ((884889.404 6539629.931, 884795.750 6..."
3,None,01005,01,84,847277,6545791,Ambérieux-en-Dombes,"POLYGON ((845951.250 6547999.078, 846073.720 6..."
4,None,01006,01,84,902191,6519791,Ambléon,"POLYGON ((901336.361 6521123.305, 902460.201 6..."


In [11]:
def codecom2(codecom1):
    if str(codecom1).startswith('75'):
        return('75056')
    elif str(codecom1).startswith('693'):
        return('69123')
    else :
        return(str(codecom1))

In [12]:
liste_med_ban['codecommunecoordstructure2']=liste_med_ban['codecommunecoordstructure'].apply(codecom2)

In [13]:
liste_med_ban_metr = liste_med_ban[liste_med_ban['Code Département']!='971']

In [14]:
df_merged_gdf = liste_med_ban.merge(gdf, how='inner', left_on='codecommunecoordstructure2', right_on='codgeo')

In [51]:
def nombre_de_med(codeCom):
    data_f = liste_med_ban_metr[(liste_med_ban_metr['codecommunecoordstructure']==codeCom)]
    return(data_f.shape[0])

liste_med_ban_metr['nb medecin commune']=liste_med_ban_metr['codecommunecoordstructure'].apply(nombre_de_med)

In [15]:
df_merged_gdf.shape

(68729, 85)

In [16]:
df_merged_gdf = df_merged_gdf[['identifiantpp', 'codecivilité',
       'nomdexercice', 'prénomdexercice', 'codecatégorieprofessionnelle',
       'codesavoirfaire', 'libellésavoirfaire', 'codemodeexercice',
       'numérosiretsite',
       'identifianttechniquedelastructur', 'raisonsocialesite',
       'numérovoiecoordstructure', 'indicerépétitionvoiecoordstructu',
       'codetypedevoiecoordstructure', 'libellétypedevoiecoordstructure',
       'libellévoiecoordstructure', 'bureaucedexcoordstructure',
       'codepostalcoordstructure', 'codecommunecoordstructure',
       'libellécommunecoordstructure', 'libellépayscoordstructure',
       'codesecteurdactivité', 'libellésecteurdactivité', 'coderôle',
       'libellérôle', 'Doctor Name', 'Specialty', 'card_0', 'card_3', 'card_4',
       'Latitude', 'Longitude', 'Skills Processed', 'RPPS', 'Nb Skills',
       'moyens_paiement', 'carte_vitale', 'secteur', 'Présentation',
       'Langues parlées', 'Diplômes nationaux et universitaires',
       'Autres formations', 'Expériences', 'Travaux et publications',
       'Prix et distinctions', 'site', 'Site web', 'Formations',
       'Diplômes nationaux et universitaires b', 'Nb caractères présentation',
       'Autres formations b', 'Travaux et publications b', 'Nb langues',
       'Expériences b', 'Formations b', 'Carac_card_1', 'Transport', 'Parking',
       'Infos', 'Coordonnées', 'Sans RDV', 'nb terms card_0',
       'nb terms card_3', 'nb terms card_4', 'nb carac card_0',
       'nb carac card_3', 'nb carac card_4', 'code dépar', 'Code Département',
       'généralistes', 'Voie', 'codgeo', 'xcl2154',
       'ycl2154']]

In [17]:
df_merged_gdf_reduit = df_merged_gdf[['identifiantpp', 'codecivilité',
       'nomdexercice', 'prénomdexercice', 'codecatégorieprofessionnelle', 'libellésavoirfaire', 'codemodeexercice', 'libellésecteurdactivité','codecommunecoordstructure',
       'libellérôle', 'Specialty', 'card_0', 'card_3', 'card_4',
       'Latitude', 'Longitude', 'Skills Processed', 'RPPS', 'Nb Skills',
       'moyens_paiement', 'carte_vitale', 'secteur', 'Présentation',
       'Langues parlées', 'Diplômes nationaux et universitaires',
       'Autres formations', 'Expériences', 'Travaux et publications',
       'Prix et distinctions', 'site', 'Site web', 'Formations',
       'Diplômes nationaux et universitaires b', 'Nb caractères présentation',
       'Autres formations b', 'Travaux et publications b', 'Nb langues',
       'Expériences b', 'Formations b', 'Carac_card_1', 'Transport', 'Parking',
       'Infos', 'Coordonnées', 'Sans RDV', 'nb terms card_0',
       'nb terms card_3', 'nb terms card_4', 'nb carac card_0',
       'nb carac card_3', 'nb carac card_4', 'Code Département',
       'généralistes', 'codgeo', 'xcl2154',
       'ycl2154']]

In [18]:
# Jointure croisée avec 'how=cross'
df_cross_0 = pd.merge(df_merged_gdf_reduit[df_merged_gdf_reduit['identifiantpp']==df_merged_gdf_reduit['identifiantpp'].iloc[1]], df_merged_gdf_reduit, how='cross')
df_cross_0.head()

,identifiantpp_x,codecivilité_x,nomdexercice_x,prénomdexercice_x,codecatégorieprofessionnelle_x,libellésavoirfaire_x,codemodeexercice_x,libellésecteurdactivité_x,codecommunecoordstructure_x,libellérôle_x,...,nb terms card_3_y,nb terms card_4_y,nb carac card_0_y,nb carac card_3_y,nb carac card_4_y,Code Département_y,généralistes_y,codgeo_y,xcl2154_y,ycl2154_y
0,10000005834,M,MARTIAL,Fabien,C,Qualifié en Médecine Générale,L,Cabinet individuel,97114,Titulaire de cabinet,...,NaN,NaN,NaN,NaN,NaN,971,150,97120,117822,6105594
1,10000005834,M,MARTIAL,Fabien,C,Qualifié en Médecine Générale,L,Cabinet individuel,97114,Titulaire de cabinet,...,NaN,NaN,NaN,NaN,NaN,971,150,97114,111368,6092163
2,10000005834,M,MARTIAL,Fabien,C,Qualifié en Médecine Générale,L,Cabinet individuel,97114,Titulaire de cabinet,...,NaN,NaN,NaN,NaN,NaN,971,150,97101,121690,6109134
3,10000005834,M,MARTIAL,Fabien,C,Qualifié en Médecine Générale,L,Cabinet individuel,97114,Titulaire de cabinet,...,NaN,NaN,NaN,NaN,NaN,971,150,97101,121690,6109134
4,10000005834,M,MARTIAL,Fabien,C,Qualifié en Médecine Générale,L,Cabinet individuel,97114,Titulaire de cabinet,...,NaN,NaN,NaN,NaN,NaN,971,150,97116,128388,6116343


In [33]:
dico_cross = []
for i in range(5000):
    df_cross = pd.merge(df_merged_gdf_reduit[(df_merged_gdf_reduit['identifiantpp']==df_merged_gdf_reduit['identifiantpp'].iloc[i])&(df_merged_gdf_reduit['codecommunecoordstructure']==df_merged_gdf_reduit['codecommunecoordstructure'].iloc[i])], df_merged_gdf_reduit[df_merged_gdf_reduit['Code Département']==df_merged_gdf_reduit['Code Département'].iloc[i]], how='cross')
    dico_d={}
    df_cross['distance']=0
    codegeo1 = df_cross['codgeo_x'].iloc[0]
    Latitude1= df_cross['xcl2154_x'].iloc[0]
    Longitude1=  df_cross['ycl2154_x'].iloc[0]
    for i in range(df_cross.shape[0]):
        codegeo2 = df_cross['codgeo_y'].iloc[i]
        if codegeo1==codegeo2:
            None
        elif (codegeo2,codegeo1) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo2,codegeo1)]
        elif (codegeo1,codegeo2) in dico_d :
            df_cross.loc[i,'distance']=dico_d[(codegeo1,codegeo2)]
        else :
            distance12 = distance_bis(Latitude1,Longitude1,df_cross_0['xcl2154_y'].iloc[i], df_cross_0['ycl2154_y'].iloc[i])
            dico_d[(codegeo1,codegeo2)]=distance12
            df_cross.loc[i,'distance']=distance12
    ligne = nbDocDist([2,8,30],df_cross)
    dico_cross.append(ligne)
    print(i)

/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_10862/3428777498.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '14.90120387753956' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_cross.loc[i,'distance']=distance12
/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_10862/3428777498.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '14.90120387753956' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_cross.loc[i,'distance']=distance12
/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_10862/3428777498.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.2433790631614645' has dtype incompatible with int64, please explicitly

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_10862/3428777498.py", line 21, in <module>
    ligne = nbDocDist([2,8,30],df_cross)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_10862/2547524388.py", line 6, in nbDocDist
    if data['distance'].iloc[i]<distanceMax :
       ~~~~~~~~~~~~~~~~~~~~~^^^
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/pandas/core/indexing.py", line 1191, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/julie/miniconda3/lib/python3.11/site-packages/pandas/core/indexing.py", line 1754, in _getitem_axis
    return self.obj._ixs(key, axis=axis)
           ^^^^^^^^^^^^^^^^^^

In [ ]:
dico_cross = []
for i in range(5000):
    df_cross = pd.merge(df_merged_gdf_reduit[(df_merged_gdf_reduit['identifiantpp']==df_merged_gdf_reduit['identifiantpp'].iloc[i])&(df_merged_gdf_reduit['codecommunecoordstructure']==df_merged_gdf_reduit['codecommunecoordstructure'].iloc[i])], df_merged_gdf_reduit[df_merged_gdf_reduit['Code Département']==df_merged_gdf_reduit['Code Département'].iloc[i]], how='cross')
    dico_cross.append(df_cross)
    if i%100==0:
        print(i)

In [ ]:
for i in range(5000,10000):
    df_cross = pd.merge(df_merged_gdf[(df_merged_gdf['identifiantpp']==df_merged_gdf['identifiantpp'].iloc[i])&(df_merged_gdf['codecommunecoordstructure']==df_merged_gdf['codecommunecoordstructure'].iloc[i])], df_merged_gdf[df_merged_gdf['Code Département']==df_merged_gdf['Code Département'].iloc[i]], how='cross')
    dico_cross.append(df_cross)
    if i%100==0:
        print(i)

In [ ]:
for i in range(10000,20000):
    df_cross = pd.merge(df_merged_gdf[(df_merged_gdf['identifiantpp']==df_merged_gdf['identifiantpp'].iloc[i])&(df_merged_gdf['codecommunecoordstructure']==df_merged_gdf['codecommunecoordstructure'].iloc[i])], df_merged_gdf[df_merged_gdf['Code Département']==df_merged_gdf['Code Département'].iloc[i]], how='cross')
    dico_cross.append(df_cross)
    #print(i)

In [ ]:
for i in range(20000,30000):
    df_cross = pd.merge(df_merged_gdf[(df_merged_gdf['identifiantpp']==df_merged_gdf['identifiantpp'].iloc[i])&(df_merged_gdf['codecommunecoordstructure']==df_merged_gdf['codecommunecoordstructure'].iloc[i])], df_merged_gdf[df_merged_gdf['Code Département']==df_merged_gdf['Code Département'].iloc[i]], how='cross')
    dico_cross.append(df_cross)
    print(i)

In [ ]:
for i in range(35000,40000):
    df_cross = pd.merge(df_merged_gdf[(df_merged_gdf['identifiantpp']==df_merged_gdf['identifiantpp'].iloc[i])&(df_merged_gdf['codecommunecoordstructure']==df_merged_gdf['codecommunecoordstructure'].iloc[i])], df_merged_gdf[df_merged_gdf['Code Département']==df_merged_gdf['Code Département'].iloc[i]], how='cross')
    dico_cross.append(df_cross)
    print(i)

In [ ]:
for i in range(40000,45000):
    df_cross = pd.merge(df_merged_gdf[(df_merged_gdf['identifiantpp']==df_merged_gdf['identifiantpp'].iloc[i])&(df_merged_gdf['codecommunecoordstructure']==df_merged_gdf['codecommunecoordstructure'].iloc[i])], df_merged_gdf[df_merged_gdf['Code Département']==df_merged_gdf['Code Département'].iloc[i]], how='cross')
    dico_cross.append(df_cross)
    print(i)

In [ ]:
for i in range(45000,50000):
    df_cross = pd.merge(df_merged_gdf[(df_merged_gdf['identifiantpp']==df_merged_gdf['identifiantpp'].iloc[i])&(df_merged_gdf['codecommunecoordstructure']==df_merged_gdf['codecommunecoordstructure'].iloc[i])], df_merged_gdf[df_merged_gdf['Code Département']==df_merged_gdf['Code Département'].iloc[i]], how='cross')
    dico_cross.append(df_cross)
    print(i)

In [ ]:
for i in range(50000,55000):
    df_cross = pd.merge(df_merged_gdf[(df_merged_gdf['identifiantpp']==df_merged_gdf['identifiantpp'].iloc[i])&(df_merged_gdf['codecommunecoordstructure']==df_merged_gdf['codecommunecoordstructure'].iloc[i])], df_merged_gdf[df_merged_gdf['Code Département']==df_merged_gdf['Code Département'].iloc[i]], how='cross')
    dico_cross.append(df_cross)
    print(i)

In [ ]:
for i in range(55000,60000):
    df_cross = pd.merge(df_merged_gdf[(df_merged_gdf['identifiantpp']==df_merged_gdf['identifiantpp'].iloc[i])&(df_merged_gdf['codecommunecoordstructure']==df_merged_gdf['codecommunecoordstructure'].iloc[i])], df_merged_gdf[df_merged_gdf['Code Département']==df_merged_gdf['Code Département'].iloc[i]], how='cross')
    dico_cross.append(df_cross)
    print(i)

In [ ]:
for i in range(60000,65000):
    df_cross = pd.merge(df_merged_gdf[(df_merged_gdf['identifiantpp']==df_merged_gdf['identifiantpp'].iloc[i])&(df_merged_gdf['codecommunecoordstructure']==df_merged_gdf['codecommunecoordstructure'].iloc[i])], df_merged_gdf[df_merged_gdf['Code Département']==df_merged_gdf['Code Département'].iloc[i]], how='cross')
    dico_cross.append(df_cross)
    print(i)

In [ ]:
df_cross_0 = pd.read_csv('df_cross_0.csv')

In [22]:
def distance_bis(x1,y1,x2,y2):
    return((np.sqrt((x1-x2)**2+(y1-y2)**2))/1000)

In [ ]:
dico_d={}
df_cross_0['distance']=0
codegeo1 = df_cross_0['codgeo_x'].iloc[0]
Latitude1= df_cross_0['xcl2154_x'].iloc[0]
Longitude1=  df_cross_0['ycl2154_x'].iloc[0]
for i in range(df_cross_0.shape[0]):
    codegeo2 = df_cross_0['codgeo_y'].iloc[i]
    if codegeo1==codegeo2:
        None
    elif (codegeo2,codegeo1) in dico_d :
        df_cross_0.loc[i,'distance']=dico_d[(codegeo2,codegeo1)]
    elif (codegeo1,codegeo2) in dico_d :
        df_cross_0.loc[i,'distance']=dico_d[(codegeo1,codegeo2)]
    else :
        distance12 = distance_bis(Latitude1,Longitude1,df_cross_0['xcl2154_y'].iloc[i], df_cross_0['ycl2154_y'].iloc[i])
        dico_d[(codegeo1,codegeo2)]=distance12
        df_cross_0.loc[i,'distance']=distance12

df_cross_0['distance'].describe()

In [ ]:
df_cross_0[['codepostalcoordstructure_x','codepostalcoordstructure_y','distance']]

In [32]:
def nbDocDist(listeDist,data):
    longueur = data.shape[0]
    nbListe = [0 for i in range(len(listeDist))]
    for (j,distanceMax) in enumerate(listeDist) : 
        for i in range(longueur):
            if data['distance'].iloc[i]<distanceMax :
                if data['RPPS_y'].iloc[i]>0 :
                    nbListe[j]=nbListe[j]+1
        data['Nb docteur à '+str(distanceMax)+' km']=nbListe[j]
    return data[['identifiantpp_x','codecivilité_x','nomdexercice_x','prénomdexercice_x','codecatégorieprofessionnelle_x','libellésavoirfaire_x','codemodeexercice_x','libellésecteurdactivité_x','libellérôle_x','Specialty_x', 'card_0_x', 'card_3_x', 'card_4_x',
       'Latitude_x', 'Longitude_x', 'Skills Processed_x', 'RPPS_x','Nb Skills_x','moyens_paiement_x', 'carte_vitale_x', 'secteur_x', 'Présentation_x',
       'Langues parlées_x', 'Diplômes nationaux et universitaires_x',
       'Autres formations_x', 'Expériences_x', 'Travaux et publications_x',
       'Prix et distinctions_x','site_x', 'Site web_x', 'Formations_x',
       'Diplômes nationaux et universitaires b_x',
       'Nb caractères présentation_x', 'Autres formations b_x',
       'Travaux et publications b_x', 'Nb langues_x', 'Expériences b_x',
       'Formations b_x','Carac_card_1_x', 'Transport_x', 'Parking_x', 'Infos_x',
       'Coordonnées_x', 'Sans RDV_x', 'nb terms card_0_x', 'nb terms card_3_x',
       'nb terms card_4_x', 'nb carac card_0_x', 'nb carac card_3_x',
       'nb carac card_4_x', 'codgeo_x', 'xcl2154_x', 'ycl2154_x','Nb docteur à 2 km','Nb docteur à 8 km','Nb docteur à 30 km']].iloc[0]           
        
#liste_0 =nbDocDist([2,8,30],df_cross_0)
#print(liste_0)

#KeyError: "['codesavoirfaire_x', 'numérosiretsite_x', 'identifianttechniquedelastructur_x', 'raisonsocialesite_x', 'code dépar_x', 'Voie_x', 'Nb docteur à 2 km', 'Nb docteur à 8 km', 'Nb docteur à 30 km'] 

In [ ]:
df_cross_0.head()

In [ ]:
print(df_cross_0[df_cross_0['RPPS_y']>0][['RPPS_y','distance']],df_cross_0[df_cross_0['RPPS_y']>0].shape)

### API OpenRouteService

In [ ]:
import openrouteservice

# Clé API OpenRouteService
client = openrouteservice.Client(key='5b3ce3597851110001cf6248fa09b3e419ef40e182777370274818e9')

# Coordonnées (longitude, latitude) pour le départ et l'arrivée
coordinates = [[2.3522, 48.8566], [4.8357, 45.764]]

# Calcul de l'itinéraire
route = client.directions(coordinates, profile='driving-car')

# Temps de trajet en secondes
duration = route['routes'][0]['summary']['duration']
print(f"Temps de trajet : {duration / 60} minutes")